In [0]:
import base64
import os
import numpy as np
import pandas as pd
import requests

# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - Pandas dataframe output.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.  
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    r = requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={'csv':base64.b64encode(data.to_csv(index=False).encode('ascii')).decode("ascii"),
        'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code == 200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))
        

# These functions are provided to build a submission dataframe from the two images that you must
# generate for this assignment.  It is unlikely that you would need to modify these.
def scan_pixel(d,img_array,img_num,pix_num,x,y):
    d[f'img{img_num}-p{pix_num}-rgb0'] = [img_array[y,x,0]]
    d[f'img{img_num}-p{pix_num}-rgb1'] = [img_array[y,x,1]]
    d[f'img{img_num}-p{pix_num}-rgb2'] = [img_array[y,x,2]]

def scan_image(d,img_num,img):
    img_array = np.asarray(img)
    rows = img_array.shape[0]
    cols = img_array.shape[1]
    d[f'img{img_num}-height'] = [rows]
    d[f'img{img_num}-width'] = [cols]
    scan_pixel(d,img_array,img_num,0,0,0)
    scan_pixel(d,img_array,img_num,1,int(cols/2),int(rows/2))
    scan_pixel(d,img_array,img_num,2,cols-1,rows-1)

def build_submit(submit_img1, submit_img2):
    d = {}
    scan_image(d,1,submit_img1)
    scan_image(d,2,submit_img2)
    return pd.DataFrame(d)

In [0]:
5//2

In [0]:
%matplotlib inline

import os
import pandas as pd
import io
import requests
import numpy as np
from scipy.stats import zscore
from PIL import Image, ImageFile
from matplotlib.pyplot import imshow
import requests
from io import BytesIO
import numpy as np

# This is your student key that I emailed to you at the beginnning of the semester.


# You must also identify your source file.  (modify for your local setup)
# file='/resources/t81_558_deep_learning/assignment_yourname_class6.ipynb'  # IBM Data Science Workbench
# file='C:\\Users\\jeffh\\projects\\t81_558_deep_learning\\t81_558_class6_intro_python.ipynb'  # Windows


def fill_square_image(img):
    # ************* Add your solution here*********
    # You should not need to modify the other code.  
    # The return statement should be replaced with your own.
    img = np.array(img)
    #print(img.shape)
    H, W, C = img.shape
    avg_r = int(np.sum(img[:,:,0])/(H*W))
    avg_g = int(np.sum(img[:,:,1])/(H*W))
    avg_b = int(np.sum(img[:,:,2])/(H*W))
    #print(avg)
    
    square = np.zeros((H,H,3)) if H > W else np.zeros((W,W,3))
    
    if H > W:
        diff = H -W
        if diff % 2 == 0:
            left = diff // 2
            right = diff // 2
            square[:,0:left,0] = avg_r
            square[:,left+W:,0] = avg_r
            square[:,0:left,1] = avg_g
            square[:,left+W:,1] = avg_g
            square[:,0:left,2] = avg_b
            square[:,left+W:,2] = avg_b
            square[:,left:left+W,:] = img
        else:
            left = diff // 2
            right = diff // + 1
            square[:,0:left,0] = avg_r
            square[:,left+W:,0] = avg_r
            square[:,0:left,1] = avg_g
            square[:,left+W:,1] = avg_g
            square[:,0:left,2] = avg_b
            square[:,left+W:,2] = avg_b
            square[:,left:left+W,:] = img
            
    if H < W:
        diff = W - H
        if diff % 2 == 0:
            up = diff // 2
            down = diff // 2
            square[0:up,:,0]= avg_r
            square[up+H:,:,0] = avg_r
            square[0:up,:,1] = avg_g
            square[up+H:,:,1] = avg_g
            square[0:up,:,2] = avg_b
            square[up+H:,:,2] = avg_b
            square[up:up+H,:,:] = img
        else:
            up = diff // 2
            down = diff // 2 + 1
            square[0:up,:,0] = avg_r
            square[up+H:,:,0] = avg_r
            square[0:up,:,1] = avg_g
            square[up+H:,:,1] = avg_g
            square[0:up,:,2] = avg_b
            square[up+H:,:,2] = avg_b
            square[up:up+H,:,:] = img
            
    return Image.fromarray(np.uint8(square))

# Handle first image
url = "https://github.com/jeffheaton/t81_558_deep_learning/raw/master/photos/hickory_home.jpg"

response = requests.get(url)
img = Image.open(BytesIO(response.content))
img.load()
submit_img1 = fill_square_image(img)
display(submit_img1)

In [0]:
# Handle second image
url = "https://github.com/jeffheaton/t81_558_deep_learning/raw/master/photos/landscape.jpg"

response = requests.get(url)
img = Image.open(BytesIO(response.content))
img.load()
submit_img2 = fill_square_image(img)
display(submit_img2)

In [0]:
# Submit
submit_df = build_submit(submit_img1, submit_img2)
submit(source_file=file,data=submit_df,key=key,no=6)